## Projet Machine learning : emission de CO2 des véhicules commercial en France 


In [1]:
import pandas as pd
df = pd.read_csv("data/CO2csv.csv", sep=";", encoding="latin1")
print("\nLe jeu de donnée à été charger avec succes !\n")

# Aperçu des données
#print('\n',df.head())
print(' ')

# Informations sur les colonnes
print(df.info())

# Statistiques descriptives
print('\n',df.describe())

# Dimensions du dataset
print('\n',f"Dimensions : {df.shape}")
print('')

print('\n',df.describe(include='object'))


Le jeu de donnée à été charger avec succes !

 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55044 entries, 0 to 55043
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lib_mrq            55044 non-null  object 
 1   lib_mod_doss       55044 non-null  object 
 2   lib_mod            55044 non-null  object 
 3   dscom              55044 non-null  object 
 4   cnit               55044 non-null  object 
 5   tvv                55044 non-null  object 
 6   cod_cbr            55044 non-null  object 
 7   hybride            55044 non-null  object 
 8   puiss_admin_98     55044 non-null  int64  
 9   puiss_max          55044 non-null  object 
 10  typ_boite_nb_rapp  55044 non-null  object 
 11  conso_urb          55001 non-null  object 
 12  conso_exurb        55001 non-null  object 
 13  conso_mixte        55010 non-null  object 
 14  co2                55010 non-null  float64
 15  co_typ_1           54

## analyse des données manquante et partie netoyage

### suppression des colonne avec trop de val manquante

In [3]:
# suppression de column inutile

print(df.isna().sum())
df.drop(['Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29'], axis=1, inplace=True)
print(list(df.columns))
#suppression de date_maj car trop de donnée manquante et difficulté majeur a emputé les donnée (92.7% manquant)
# suppression de HC, HCNOX car il on beaucoup de val manquante et que hcNox depend de nox et de hc
df.drop(['date_maj','hc', 'hcnox'], axis=1, inplace=True)
print(list(df.columns))



lib_mrq                  0
lib_mod_doss             0
lib_mod                  0
dscom                    0
cnit                     0
tvv                      0
cod_cbr                  0
hybride                  0
puiss_admin_98           0
puiss_max                0
typ_boite_nb_rapp        0
conso_urb               43
conso_exurb             43
conso_mixte             34
co2                     34
co_typ_1               158
hc                   45271
nox                    158
hcnox                 9888
ptcl                  2678
masse_ordma_min          0
masse_ordma_max          0
champ_v9                61
date_maj             51797
Carrosserie              0
gamme                    0
Unnamed: 26          55044
Unnamed: 27          55044
Unnamed: 28          55044
Unnamed: 29          55044
dtype: int64
['lib_mrq', 'lib_mod_doss', 'lib_mod', 'dscom', 'cnit', 'tvv', 'cod_cbr', 'hybride', 'puiss_admin_98', 'puiss_max', 'typ_boite_nb_rapp', 'conso_urb', 'conso_exurb', 'conso_mixte

> verification des type des colonne car j'ai remarque que certaine colonne etait object et non float :
>
- nox est un object
- ptcl object
- conso_urb
- conso_ex
- conso_mix

In [1]:
#verification du type de la colonne nox
print(df['nox'].dtype)
print(df['ptcl'].dtype)
print(df['conso_mixte'].dtype)
print(df['conso_urb'].dtype)
print(df['conso_exurb'].dtype)

NameError: name 'df' is not defined

In [6]:
# Imputation des variables conservées
# imputations par median des valeur afin de garde le max de val

#df['nox'].fillna(df['nox'].median(), inplace=True)
#df['ptcl'].fillna(df['ptcl'].median(), inplace=True)
#df['conso_urb'].fillna(df['conso_urb'].mean(), inplace=True)
#df['conso_ex'].fillna(df['conso_ex'].mode(), inplace=True)
#df['conso_mixte'].fillna(df['conso_mixte'].mode(), inplace=True)
#df['co2'].fillna(df['co2'].mode(), inplace=True)
#df['co_typ_1'].fillna(df['co_typ_1'].mode(), inplace=True)
print(df.isna().sum())

lib_mrq                 0
lib_mod_doss            0
lib_mod                 0
dscom                   0
cnit                    0
tvv                     0
cod_cbr                 0
hybride                 0
puiss_admin_98          0
puiss_max               0
typ_boite_nb_rapp       0
conso_urb              43
conso_exurb            43
conso_mixte            34
co2                    34
co_typ_1              158
nox                   158
ptcl                 2678
masse_ordma_min         0
masse_ordma_max         0
champ_v9               61
Carrosserie             0
gamme                   0
dtype: int64


## what do i do about the object colonnes mdr

In [ ]:
#verification du type de la colonne nox
print(df['nox'].dtype)